# Introduction 
This is an example of a CANOPUS notebook for visualizing and analyzing compound classes.

First we have to load the canopus package. I wrapped all important stuff into the Canopus class, so the easiest way to work with that (and a somewhat stable api) is to use just this single class.

I also increase the default figure size of matplotlib, just for the case that your default rc parameters are as bad as mine.

In [ ]:
import canopus
from canopus import Canopus
import matplotlib.pyplot as plt
import seaborn as sbn
sbn.set_context("notebook")
%matplotlib inline
plt.rc("figure", figsize=(14,5))

# Processing the data

We will demonstrate the whole processing on a small example of the rosmarine dataset from https://massive.ucsd.edu/ProteoSAFe/dataset.jsp?task=9a75f69f0f33460293ee3928361ab836.

1. Download all mzml files via ftp download from the Massive dataset. We store the mzml files in the directory rosmarin_mzml
2. Process the data with the SIRIUS 4.4

`sirius -i rosmarin_mzml -o sirius_rosmarin --maxmz=800 lcms-align sirius zodiac fingerid canopus`
- The `--maxmz` option restricts the input data to small compounds. Instead of 800 you can also use smaller or larger values, but computing time of SIRIUS increase exponentially with the mass of the compounds.
- The `lcms-align` option is only necessary if you have multiple LCMS-runs and you want to align them
- This will take some time, but in the end you get the results written into the sirius_rosmarin folder


3. Now we want to process the same data with GNPS (this is optional). For this we export the sirius projectspace as MGF file which can be read by GNPS. Furthermore, we export a quantification file which can be used with ion identity networking.

`sirius -i sirius_rosmarin mgf-export --merge-ms2  --quant-table=rosmarin-quant.csv --output rosmarin.mgf`

4. You can now upload the rosmaring.mgf and rosmarin-quant.csv file to GNPS and start ion identity molecular networking. Download the molecular networking data and extract the archive into a directory with name gnps_rosmarin

Now we can start with the Jupyter notebook analysis. We initialize the Canopus object with the name of the SIRIUS and GNPS folders. I use the short variable name C, because we will use this variable a lot.


In [ ]:
C = Canopus(sirius="sirius_rosmarin", gnps="gnps_rosmarin")

# CANOPUS data analysis

## Defining the metadata

The dataset consists of samples and features. The samples can be usually organized in several groups. We call these groups "conditions", but they can be treatments, several tissues, organs, different species and so on. We will first tell CANOPUS which conditions in your data exist and how you can map these conditions to the file names of the samples.

The defineCondition method gets three parameters: first the name of the condition, then a regular expression to detect the filenames of samples having this condition, and finally a color which should be used for this condition in plots. We can leave out the second and third parameter: without a regular expression, all filenames containing the condition name as substring belong automatically to this condition and the color is given automatically, too.

In [ ]:
C.defineCondition("Leave", ".*_Leave.*", "brown")
# we can also leave out parameters
C.defineCondition("Flower", color="steelblue")
C.defineCondition("Stem");

To check if we did something wrong, we can output the list of files belonging to a condition

In [ ]:
C.condition("Flower").samples

## Displaying single compounds

Each feature (or compound) has an ID which is a string (although it is usually numerical). We can list all compounds with the following:

In [ ]:
C.compounds

If we want to know more about a certain compound, we can use the *featureHeader* method which just tells us the molecular formula identification of the feature, its adduct as well as its ZODIAC score.

In [ ]:
C.featureHeader("121")

With gnpsHit we get the result of the GNPS database search for this hit.

In [ ]:
C.gnpsHit(121);

With identification we get the CSI annotation for this hit.

In [ ]:
C.identification("121");

With classification we get the compound classification of this compound.

It outputs the tree with the categories and their probabilities. Categories with high probability are in bold. You can hover with the mouse over the categories to get a tooltip about their meaning.

In [ ]:
C.classification("121")

Finally, we might be interested in which samples this feature appears. This can be done with histogram.

In [ ]:
plt.rc("figure", figsize=(14,5))
C.histogram("121")

There are always two plots: Left is the plot of normalized intensities. The normalization is done by:

1. Apply Quantile Normalization. This kind of normalization gives the highest compound in each sample the same intensity, as well as the second highest and so on.
2. Normalize each sample individually, such that the highest compound gets intensity 1.

The resulting quant table can be accessed with `C.QuantNormalized`

There is a second normalization routine which happens after the first one:

1. Add a pseudocount on all intensities (use the 1% quantile of intensities for that)
2. Take the logarithm with base 10 on all intensities
3. Round all values to the next 0.5 number. This results in a very discrete looking dataset where small intensity deviations are removed from data.

The result of the second normalization step can be accessed with `C.QuantLogarithmic`

The plot above shows both normalizations.

We can repeat all steps above in a single command:


In [ ]:
C.describe("380")

In [ ]:
C.gnps_hits

# More about  conditions

Before we continue with the next topic, let me give a short explanation about conditions and grouping conditions. We have seen in the last plots, that all conditions we have defined so far are represented in the plot. We can also give a list of conditions to display, though:

In [ ]:
C.histogram("380", ["Flower", "Stem"])

There are two additional methods: **join** and **select**. With join we group conditions together and say "one of these conditions have to be true".

In [ ]:
C.histogram("380", ["Flower", C.join("Leave", "Stem")])

In this example it does not make much sense to join the two conditions Leave and Stem. But when we have A LOT of conditions, we might want to plot only a subset of them. Just an example: the mice dataset has two conditions for the treatment (GF and SPF) as well as many conditions for the organs (Mouth, Stomach, and so on). If we want to simplify the plot by merging all organs, which are part of the guts, together, we can use the join method for that. We can also assign a name and a color to it:

In [ ]:
C.histogram("380", ["Flower", C.join("Leave", "Stem",name = "green parts", color="seagreen")])

While **join** is taking the union of all samples of different conditions, **select** is taking the intersection. It does not make sense for this dataset, because we have only distinguished conditions. But for the mouse dataset we could define a group for all samples that are from the germ free mouse and from the guts. We could define such a set as:
`C.select("GF",C.join("Duo","Jeju","Ile","Cecum","Colon"))`

## Displaying a complete dataset

The Sunburst plot allows for visualizing a complete dataset. We can optionally provide a condition for visualization, or just visualize everything. 

In [ ]:
C.treemap()

In [ ]:
C.allFromCategory(C["Fatty acid esters"])

## Differential analysis

The idea of this kind of analysis is that we want to know, which compound classes are important to differentiate between two conditions. For example: what are the compound classes that are present in Leave but absent in Stem or vice versa.

For finding compound classes which are important to differentiate between the conditions, we proceed the following:

1. order all compounds according to how good they discriminate between the conditions
2. make a permutation test to find out if many of these high discriminating compound classes belong to the same compound class

Because this kind of analysis is quite complex, we first build a new object for differential analysis by calling the differential method. This method gets the two conditions as parameters. If we left out the second condition, all conditions except the first one is used automatically. We save the return value in the variable D.

The last parameter is the ordering method. by default, it is "robust_forest", which is a random forest on the logarithmic data. We can also choose other methods, but let us keep by the default first.

In [ ]:
D = C.differential("Leave", "Stem")

## Discriminating compounds

The first thing we might be interested in is the ordering of the compounds. What are the most discriminating compounds.

In [ ]:
D.topCompounds();



The weight column is the feature importance value defined by the random forest. The category column displays **ONE** compound category of the compound. Note, that there might be (and are) other categories, because each compound can have many categories at the same time.

We should check if these compounds are really discriminating:


In [ ]:
C.histogram("488", ["Leave", "Stem"])

I think this shows quite nice, that the first compound is, indeed, very strongly expressed in leaves but not in stems. We can get more information about this compound as usual. For example with the second compound:

In [ ]:
C.describe(488)

This compound is not found in the GNPS spectral libraries. That's why the GNPS search results are empty. We still get useful CANOPUS annotations.

## Discriminating compound categories

Now we want to know: what are the most differentiating compound categories.

In [ ]:
D.topCategories();

It is the same idea, this time with compound categories. The p-value column is the output of the permutation test (I haven't added a correction, though, so don't take this value serious).

We might now wonder if Methyl esters are really differential expressed in both classes. We can use heatmaps to find out:

In [ ]:
C.heatmap("Methyl esters")

The rows in this plot are the compounds, the columns are the samples. The color coding goes from low intensity (dark blue) to high intensity (yellow).

Okay, we see that there are many compounds in Stem belonging to the Methyl esters class and which are lower expressed in leave and vice versa. We also see, that one compound is very highly expressed in leave.

What are the other methods for discrimination? First, we can use random forest on the normalization data using the "forest" keyword. With the "fold_change" keyword we look for classes which are differential expressed just by looking at the trimmed mean fold change between the samples.

In [ ]:
D = C.differential("Flower", method="fold_change")

In [ ]:
D.topCategories();

In [ ]:
C.heatmap("Benzene and substituted derivatives")

With **differentialTreemap** we can draw two sunburst plots which compare two conditions.

In [ ]:
C.differentialTreemap("Flower", "Leave")

## Using autocompletion for compound names, category names and conditions

If you want to know the expression of a certain category, you can use the autocompletion of Jupyter notebook. Let us assume we want to make a heatmap of flavonoids, but we do not exactly know how this category is written. We can just write

`C["Fl`

and then press [TAB] and the remaining name is autocompleted. This is also working with conditions and compound ids.

# Molecular Network analysis

In the following we take a look how we can take advantage from the GNPS molecular networking information.

First, we can query the GNPS hits by compound categories (although that is not very exciting):

In [ ]:
C.gnpsHit(category="Flavonoids");

Note that the query is using the canopus annotations for the category. Thus, all GNPS hits in this table are annotated by CANOPUS as Flavonoids.

Now, let us take a look into the molecular networks themself:

In [ ]:
C.molecularNetwork()

The first view shows the complete network, but it is currently too large to be useful. So we should click first on the Display subnetwork button below. Now we can scroll through all the GNPS networks for this dataset. The majority category above tells us which compound category is dominant in this cluster. Furthermore, all nodes belonging to this category are highlighted red. We can also click on a node and see its classification.

If we click on the name of any compound class, the highlighting changes such that we see which nodes belong to this class. We can inspect the original network by clicking on the view network in GNPS link below.

Note that the Majority Class is currently very primitive: it just uses the class deepest in the tree which occurs for more than 50% of the network. However, often this is not the most interesting class.

GNPS hits are highlighted in orange. So, click through the network to find a cluster which is not annotated yet! This might be the interesting one where canopus annotation can help.


## Explore data with Cytoscape

The builtin network viewer is rather simple. For a better exploration of the network we recommend the Cytoscape software. To convert the CANOPUS results into a graphml file, use the **exportCytoscape** function. We can call this function with a list of manually selected class names, or let CANOPUS decide which classes might be interesting for annotation. GraphML files are table based, while ClassyFire annotations are a directed acyclic graph. So we need some way for assigning single classes to a node, although reality is more complex. For best visualuation results we recommend to manually assign useful compound classes.

In [ ]:
C.exportCytoscape("file1.graphml")
#C.exportCytoscape("file2.graphml", [C["Prenol lipids"], C["Flavonoids"], C["Amino acids, peptides, and analogues"], 
#                                    C["Benzene and substituted derivatives"]])

## NPC Classifier

The latest version of CANOPUS also provides predictions for the NPC (Natural Product Classifier). See http://classifier.ucsd.edu/ and the preprint at [arxiv](https://chemrxiv.org/articles/preprint/NPClassifier_A_Deep_Neural_Network-Based_Structural_Classification_Tool_for_Natural_Products/12885494/1).

In [ ]:
C.npcClassification(100)

Compared to the ClassyFire ontology, the NPC usually assigns only a single class to a compound. This provides less information, but makes analysis and evaluation easier.
We recommend to always look at both, NPC and ClassyFire predictions. 

Note that the NPC has a special category "glycosylated" which is not predicted by CANOPUS. Instead, just refer to the Classyfire class **Glycosyl compounds**

In [ ]:
C.classification(100)

You can create a Pandas Dataframe with all NPC categories and their probabilities. We also add the most specific ClassyFire class to the dataframe for comparison:

In [ ]:
C.npcSummary()

The datafame can be saved in CSV format and opened with Excel.

In [ ]:
C.npcSummary().to_csv("npc_summary.csv")